# 螺旋桨RNA UPP预测 #

In [1]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
# !mkdir /home/aistudio/external-libraries
# !pip install beautifulsoup4 -t /home/aistudio/external-libraries

In [2]:
# 程序在/work/code目录下， 需先加入路径
import sys 
sys.path.append('/home/aistudio/work/code')
# fly_paddle是唯一需要直接调用的模块
# fly_paddle is the only module required for interactive sessions
import fly_paddle as fp

# args包括几乎所有需要的参数， 贯穿于几乎所有的程序调用中
# args由fp.parse_args2()根据任务初始化, 要用到的任务包括： ‘train', 'validate', 'predict'
# args is a structure storing most (if not all) parameters and used for most function calls.
# args is initialized by fp.parse_args2(), depending on the specific task, such as "train", "validate" "predict"
args, _ = fp.parse_args2('train')
print(fp.gwio.json_str(args.__dict__))
# 注： 根据不同的网络等等需要， args可能包含一些用不到的参数
# Attention: some parameters in args may not be used depending on the network etc.

{
   "action": "train",
   "argv": "-h",
   "verbose": 1,
   "resume": false,
   "load_dir": null,
   "save_dir": null,
   "save_level": 2,
   "save_grpby": ["epoch", "batch"],
   "log": "fly_paddle-May13.log",
   "data_args": "======= data args =======",
   "data_dir": "data",
   "data_name": "predict",
   "data_suffix": ".pkl",
   "data_size": 0,
   "test_size": 0.1,
   "split_seed": null,
   "input_genre": "Seq",
   "input_fmt": "NLC",
   "seq_length": [0, 512, -1],
   "residue_fmt": "vector",
   "residue_nn": 0,
   "residue_dbn": false,
   "residue_attr": false,
   "residue_extra": false,
   "label_genre": "upp",
   "label_fmt": "NL",
   "label_tone": "none",
   "label_ntype": 2,
   "label_smooth": false,
   "net_args": "======= net args =======",
   "net_src_file": "/home/aistudio/work/code/paddle_nets.py",
   "net": "lazylinear",
   "resnet": false,
   "act_fn": "relu",
   "norm_fn": "none",
   "norm_axis": -1,
   "dropout": 0.2,
   "feature_dim": 1,
   "embed_dim": 32,
   "embed

In [3]:
# 两种更新args的方法： 1） args.update(**dict), 2) args.[key] = value
# Two main ways to update values in args: 1) args.update(**dict), 2) args.[key] = value
args.update(data_dir='work/data', data_name='train', residue_dbn=True, residue_extra=True)

# 网络参数 （net parameters): 
# 网络的设计主要考虑了三个支配RNA碱基配对的因素： 
#    1) 来自于全部序列的排列组合（配分）竞争，用Attention机制来模拟
#    2）来自于线性大分子的一维序列限制， 用LSTM结构来模拟
#    3）来自于局部紧邻碱基的合作（比如，一个孤立的碱基对极不稳定）， 用1D Convolution来模拟
# 所以框架由以上三个模块组成， 并在输入和输出层加了1-3个线性层。除非特意说明， 所有的隐藏层的维度为32.
# 训练中发现高维度和深度的网络并不能给出更好的结果！
# Three main mechanisms directing RNA base pairing are taken into consideration for the 
# design of the network architecture. 
#   1) The combinatorial configurational space of attainable RNA base pairs, approximated by Attention Mechanism
#   2) The quasi-1D nature of unbranched, continuous RNA polymers, approximated by LSTM
#   3) The cooperativity of neighboring bases for stable base pairing, approximated by 1D Convolution
# Hence the neural net comprises of three main building blocks, with additional linear layers for the input and output. 
# The dimensions of all hidden layers are 32 unless noted otherwise.
# Larger and/or deeper nets gave similar, but no better, performances!
args.net='seq2seq_attnlstmconv1d'  # the net name defined in paddle_nets.py
# 输入模块由一个线性层组成
# The input block is a single linear feedforward layer
args.linear_num = 1 # the number of linear feedforward layers
# 三大处理模块 (the three main data-crunching blocks)
args.attn_num = 1 # the number of transformer encoder layers
args.lstm_num = 1 # the number of bidirectional lstm layers
args.conv1d_num = 1 # the number of 1D convolution layers
# 输出模块由三个线性层组成， 维度分别为32, 32, 2
# three linear layers for the final output, with dimensions of 32, 32, and 2, respectively
args.output_dim = [32, 32, 2]
# 如果序列被补长到同一长度， 对归一化的影响不清楚， 所以用batch_size=1
# If sequences are padded to the same length, such padding may interfere with normalization, hence batch_size=1 
args.norm_fn = 'layer' # layer normalization
args.batch_size = 1 # 1 is used in consideration of the layer norm above
# 最后递交用的损失函数选为softmax+bce, 也可以用 softmax+mse, 结果几乎一样
# The submitted results were trained with softmax+bce loss function. 
# Essentially the same results were obtained with softmax+mse loss function
args.loss_fn = ['softmax+bce'] # softmax is needed here as the final output has a dimension of 2
args.label_tone = 'soft' # soft label
args.loss_sqrt = True # sqrt(loss) is only necessary for softmax+mse
args.loss_padding = False # exclude padded residues from loss
# 需要运行fp.autoconfig_args()来消除参数的不一致性
# fp.autoconfig_args() is needed to resolve inconsistencies between parameters
args = fp.autoconfig_args(args)


In [4]:
# 建立和检测模型 （Get and inspect the model）
model = fp.get_model(args)
# 注： 最后的输出矩阵的维度为[N, L, 2]
# Note: the shape of the output is [N, L, 2]

2021-05-13 10:53:48,896 - INFO - Used net definition: /home/aistudio/work/code/paddle_nets.py
2021-05-13 10:53:48,986 - INFO - {'total_params': 36418, 'trainable_params': 36418}
2021-05-13 10:53:48,987 - INFO - Optimizer method: adam
2021-05-13 10:53:48,988 - INFO -    learning rate: 0.003
2021-05-13 10:53:48,988 - INFO -     lr_scheduler: reduced
2021-05-13 10:53:48,989 - INFO -     weight decay: none
2021-05-13 10:53:48,989 - INFO -          l1decay: 0.0001
2021-05-13 10:53:48,991 - INFO -          l2decay: 0.0001
2021-05-13 10:53:48,991 - INFO - Getting loss function: ['softmax+bce']


-------------------------------------------------------------------------------------------------------------------------------------
      Layer (type)                          Input Shape                                  Output Shape                   Param #    
   MyEmbeddingLayer-1                      [[2, 512, 10]]                                [2, 512, 10]                      0       
        Linear-1                           [[2, 512, 10]]                                [2, 512, 32]                     352      
         ReLU-1                            [[2, 512, 32]]                                [2, 512, 32]                      0       
       LayerNorm-1                         [[2, 512, 32]]                                [2, 512, 32]                     64       
        Dropout-1                          [[2, 512, 32]]                                [2, 512, 32]                      0       
     MyLinearTower-1                       [[2, 512, 10]]                 

In [5]:
# 读取数据. 提供的数据被转换成了一个dict, 存储为pickle文件. 
# 输入矩阵中最后两列的数据为linear_partition_c和linear_partition_v的预测结果
# read in data. The provided data are transfomed into a dict, which is saved as a pickle file
# the last two columns in the input matrix are the predictions of linear_partition_c and linear_partition_v
midata = fp.get_midata(args)
train_data, valid_data = fp.train_test_split(midata, test_size=0.1)

2021-05-13 10:53:54,736 - INFO - Loading data: work/data/train.pkl
2021-05-13 10:53:54,799 - INFO -    # of data: 5000,  max seqlen: 500, user seq_length: [0, 512, -1]
2021-05-13 10:53:54,800 - INFO -  residue fmt: vector, nn: 0, dbn: True, attr: False, genre: upp
2021-05-13 10:53:54,825 - INFO - Selected 5000 data sets with length range: [0, 512, -1]
2021-05-13 10:54:00,860 - INFO - Processing upp data...


In [6]:
# 训练模型， 最后的loss应该在[0.52, 0.53]区间内
# 每epoch需要五分钟左右(在CPU上)， 自然结束需要～20个epoch
# train the model, the final loss should be within 0.52 and 0.53.
# takes about 5 minutes to complete one epoch. 
# self-termination takes ~20 epochs
train_loss, valid_loss = fp.train(model, train_data, num_epochs=21, validate_callback = fp.func_partial(fp.validate_in_train, midata=valid_data, save_dir='./'))
# 注： 软标签的情况下不能得到0的交叉熵
# Note: zero cross-entropy is not possible with soft labels

2021-05-13 10:54:07,183 - INFO - Training, data size: 4500
2021-05-13 10:54:07,184 - INFO -          batch size: 1
2021-05-13 10:54:07,185 - INFO -             shuffle: True
2021-05-13 10:54:07,185 - INFO -        # of batches: 4500
2021-05-13 10:54:07,186 - INFO -      recap interval: 151
2021-05-13 10:54:07,186 - INFO -   validate interval: 450
2021-05-13 10:54:07,187 - INFO -         # of epochs: 21
2021-05-13 10:54:07,187 - INFO -        loss padding: False
2021-05-13 10:54:08,028 - INFO - Epoch/batch: 0/   0, ibatch:    0, loss: 0.8334, std: 0.4388
2021-05-13 10:54:16,475 - INFO - loss: 0.8304, std: 0.4166
2021-05-13 10:54:22,838 - INFO - Epoch/batch: 0/ 151, ibatch:  151, loss: 0.6754, std: 0.6512
2021-05-13 10:54:29,126 - INFO - Epoch/batch: 0/ 302, ibatch:  302, loss: 0.5804, std: 0.6641
2021-05-13 11:01:14,115 - INFO - loss: 0.5391, std: 0.6125
2021-05-13 11:01:14,636 - INFO - Epoch/batch: 1/1812, ibatch: 6312, loss: 0.5300, std: 0.6133
2021-05-13 11:01:20,789 - INFO - Epoch/b

Epoch 54: ReduceOnPlateau set learning rate to 0.0024300000000000003.


2021-05-13 11:03:08,721 - INFO - loss: 0.5360, std: 0.6108
2021-05-13 11:03:09,684 - INFO - Epoch/batch: 1/3624, ibatch: 8124, loss: 0.5452, std: 0.6181
2021-05-13 11:03:15,831 - INFO - Epoch/batch: 1/3775, ibatch: 8275, loss: 0.5598, std: 0.6385
2021-05-13 11:03:21,895 - INFO - Epoch/batch: 1/3926, ibatch: 8426, loss: 0.5343, std: 0.6129
2021-05-13 11:03:36,723 - INFO - loss: 0.5340, std: 0.6057
2021-05-13 11:03:36,741 - INFO - Saved model states in: earlystop_0.5340
2021-05-13 11:03:36,743 - INFO - Saved net python code: earlystop_0.5340/paddle_nets.py
2021-05-13 11:03:36,752 - INFO - Saved best model: earlystop_0.5340
2021-05-13 11:03:36,753 - INFO - Removing earlystop model: earlystop_0.5351
2021-05-13 11:03:37,964 - INFO - Epoch/batch: 1/4077, ibatch: 8577, loss: 0.5533, std: 0.6344
2021-05-13 11:03:44,011 - INFO - Epoch/batch: 1/4228, ibatch: 8728, loss: 0.5616, std: 0.6299
2021-05-13 11:03:50,188 - INFO - Epoch/batch: 1/4379, ibatch: 8879, loss: 0.5393, std: 0.6175
2021-05-13 11

Epoch 82: ReduceOnPlateau set learning rate to 0.002187.


2021-05-13 11:07:44,854 - INFO - Epoch/batch: 2/3322, ibatch: 12322, loss: 0.5294, std: 0.6157
2021-05-13 11:07:50,888 - INFO - Epoch/batch: 2/3473, ibatch: 12473, loss: 0.5570, std: 0.6235
2021-05-13 11:08:05,917 - INFO - loss: 0.5335, std: 0.6285
2021-05-13 11:08:06,937 - INFO - Epoch/batch: 2/3624, ibatch: 12624, loss: 0.5400, std: 0.6211
2021-05-13 11:08:12,895 - INFO - Epoch/batch: 2/3775, ibatch: 12775, loss: 0.5449, std: 0.6272
2021-05-13 11:08:19,167 - INFO - Epoch/batch: 2/3926, ibatch: 12926, loss: 0.5416, std: 0.6182
2021-05-13 11:08:35,285 - INFO - loss: 0.5328, std: 0.6016
2021-05-13 11:08:36,422 - INFO - Epoch/batch: 2/4077, ibatch: 13077, loss: 0.5526, std: 0.6340
2021-05-13 11:08:42,406 - INFO - Epoch/batch: 2/4228, ibatch: 13228, loss: 0.5318, std: 0.6078
2021-05-13 11:08:48,446 - INFO - Epoch/batch: 2/4379, ibatch: 13379, loss: 0.5086, std: 0.6004
2021-05-13 11:09:03,491 - INFO - loss: 0.5355, std: 0.6300
2021-05-13 11:09:04,551 - INFO - Epoch 2 average training loss:

...

2021-05-13 12:34:55,416 - INFO - loss: 0.5272, std: 0.6176
2021-05-13 12:34:55,671 - INFO - Epoch/batch: 20/ 906, ibatch: 90906, loss: 0.5170, std: 0.5945
2021-05-13 12:35:02,121 - INFO - Epoch/batch: 20/1057, ibatch: 91057, loss: 0.5300, std: 0.6093
2021-05-13 12:35:08,647 - INFO - Epoch/batch: 20/1208, ibatch: 91208, loss: 0.5092, std: 0.5901
2021-05-13 12:35:24,327 - INFO - loss: 0.5272, std: 0.6179
2021-05-13 12:35:24,727 - INFO - Epoch/batch: 20/1359, ibatch: 91359, loss: 0.5227, std: 0.6010
2021-05-13 12:35:30,948 - INFO - Epoch/batch: 20/1510, ibatch: 91510, loss: 0.5485, std: 0.6408
2021-05-13 12:35:36,877 - INFO - Epoch/batch: 20/1661, ibatch: 91661, loss: 0.5230, std: 0.6096
2021-05-13 12:35:52,153 - INFO - loss: 0.5267, std: 0.6120
2021-05-13 12:35:52,644 - INFO - Epoch/batch: 20/1812, ibatch: 91812, loss: 0.5288, std: 0.6013
2021-05-13 12:35:58,827 - INFO - Epoch/batch: 20/1963, ibatch: 91963, loss: 0.5228, std: 0.6009
2021-05-13 12:36:04,759 - INFO - Epoch/batch: 20/2114, 

Epoch 617: ReduceOnPlateau set learning rate to 1.717925069106706e-05.


2021-05-13 12:36:33,773 - INFO - Epoch/batch: 20/2567, ibatch: 92567, loss: 0.5118, std: 0.5949
2021-05-13 12:36:48,851 - INFO - loss: 0.5266, std: 0.6117
2021-05-13 12:36:49,588 - INFO - Epoch/batch: 20/2718, ibatch: 92718, loss: 0.5361, std: 0.6191
2021-05-13 12:36:55,771 - INFO - Epoch/batch: 20/2869, ibatch: 92869, loss: 0.5348, std: 0.6114
2021-05-13 12:37:01,712 - INFO - Epoch/batch: 20/3020, ibatch: 93020, loss: 0.5353, std: 0.6062
2021-05-13 12:37:17,022 - INFO - loss: 0.5265, std: 0.6087
2021-05-13 12:37:17,916 - INFO - Epoch/batch: 20/3171, ibatch: 93171, loss: 0.5280, std: 0.6126
2021-05-13 12:37:23,763 - INFO - Epoch/batch: 20/3322, ibatch: 93322, loss: 0.5355, std: 0.6133
2021-05-13 12:37:29,869 - INFO - Epoch/batch: 20/3473, ibatch: 93473, loss: 0.5235, std: 0.5930
2021-05-13 12:37:45,210 - INFO - loss: 0.5266, std: 0.6110
2021-05-13 12:37:46,192 - INFO - Epoch/batch: 20/3624, ibatch: 93624, loss: 0.5324, std: 0.6105
2021-05-13 12:37:52,365 - INFO - Epoch/batch: 20/3775, 

Epoch 628: ReduceOnPlateau set learning rate to 1.5461325621960354e-05.


2021-05-13 12:38:21,114 - INFO - Epoch/batch: 20/4228, ibatch: 94228, loss: 0.5238, std: 0.6005
2021-05-13 12:38:27,253 - INFO - Epoch/batch: 20/4379, ibatch: 94379, loss: 0.5276, std: 0.6061
2021-05-13 12:38:41,611 - INFO - loss: 0.5268, std: 0.6126
2021-05-13 12:38:42,732 - INFO - Epoch 20 average training loss: 0.5270 std: 0.6064
2021-05-13 12:38:42,752 - INFO - Epoch 20 average validate loss: 0.5268 std: 0.6127


In [7]:
# 读取最后一个checkpoint目录 (忽略优化器state_dict读取错误)
# read the last saved earlystop directory （ignore the error in optimizer state_dict loading)
fp.state_dict_load(model, model.validate_hist.saved_dirs[-1])
# 可以改动损失函数，检测mse损失（单个模型最好的结果： ～0.20）
# the loss_fn can be changed to softmax+mse to check the mse loss.
# (the best/lowest loss obtained from a single model was ~0.20)
args.loss_fn = ['softmax+mse']
model.loss_fn = fp.get_loss_fn(args)
valid_loss = fp.validate(model, valid_data, verbose=1, batch_size=64) # try a larger batch_size, should make no difference though

2021-05-13 12:57:33,231 - INFO - Loading model states from: earlystop_0.5264
2021-05-13 12:57:33,565 - INFO - Loaded net state: earlystop_0.5264/net.state
2021-05-13 12:57:33,570 - WARNING - Error in optim state_dict loading!
2021-05-13 12:57:33,571 - INFO - Getting loss function: ['softmax+mse']
2021-05-13 12:57:33,573 - INFO - Validating, data size: 500
2021-05-13 12:57:33,573 - INFO -            batch size: 64
2021-05-13 12:57:33,574 - INFO -               shuffle: False
2021-05-13 12:57:33,574 - INFO -          # of batches: 8
2021-05-13 12:57:33,575 - INFO -        recap interval: 1
2021-05-13 12:57:33,575 - INFO -          loss padding: False
2021-05-13 12:57:36,627 - INFO - ibatch:    0, loss: 0.2021, std: 0.2891
2021-05-13 12:57:37,426 - INFO - ibatch:    1, loss: 0.2071, std: 0.3006
2021-05-13 12:57:38,265 - INFO - ibatch:    2, loss: 0.1919, std: 0.2813
2021-05-13 12:57:39,052 - INFO - ibatch:    3, loss: 0.2279, std: 0.3225
2021-05-13 12:57:39,839 - INFO - ibatch:    4, loss

In [ ]:
# 读取预测数据， 存储预测结果
# 提交的结果是平均了三次运行最好checkpoint存储， 它们分别得到了0.24, 0.24, 0.242的sqrt(mse)损失， 平均后得到了0.238
# 可惜前两次的checkpoint没有被保存， 只保存了预测的结果.
# 虽然是同一个网络架构， 因为每次训练的train_test_split是随机的， 模型平均的效果和cross_validate相近
# Read in prediction data, and save the predicted results
# The submitted results are the average of the best checkpoints/earlystops from three independent trainings.
# Unfortunately the checkpoints for the first two were not kept, except for the predicted results。
# As each training randomly splits the train and validation data, this model averaging approximates
# the effect/benefit of cross-validation.
predict_data = fp.get_midata(args, data_name='predict', seq_length=-1)
y_model, std_model = fp.predict(model, predict_data, save_dir='predict.files', batch_size=1)

2021-05-13 12:57:54,373 - INFO - Loading data: work/data/predict.pkl
2021-05-13 12:57:55,946 - INFO -    # of data: 112,  max seqlen: 861, user seq_length: -1
2021-05-13 12:57:55,947 - INFO -  residue fmt: vector, nn: 0, dbn: True, attr: False, genre: upp
2021-05-13 12:57:58,986 - INFO - Predicting, data size: 112
2021-05-13 12:57:58,989 - INFO -            batch size: 1
2021-05-13 12:57:58,990 - INFO -               shuffle: False
2021-05-13 12:57:58,990 - INFO -          # of batches: 112
2021-05-13 12:57:58,991 - INFO -        recap interval: 4
2021-05-13 12:57:58,992 - INFO - Predicted files will be saved in: predict.files
  0%|          | 0/112 [00:00<?, ?it/s]